批量下载抱脸模型

In [ ]:
!pip install requests beautifulsoup4

In [1]:
import os
os.environ['http_proxy'] = 'http://127.0.0.1:7890'
os.environ['https_proxy'] = 'http://127.0.0.1:7890'

In [3]:
import requests
from bs4 import BeautifulSoup
import subprocess

def cmd_run(com):
    !{com}

def get_model_names_and_urls(base_url="https://huggingface.co/lllyasviel/sd_control_collection/tree/main"):
    try:
        response = requests.get(base_url)
        if response.status_code != 200:
            return "Failed to retrieve data. Status code: " + str(response.status_code)

        soup = BeautifulSoup(response.content, 'html.parser')

        # 根据新的 HTML 结构更新选择器
        links = soup.find_all('a', {'class': 'group'})

        models = []
        for link in links:
            href = link.get('href')
            if href and '/resolve/main/' in href:
                model_name = href.split('/')[-1].split('?')[0]  # 提取文件名
                download_url = "https://huggingface.co" + href  # 构造完整的下载链接
                models.append((model_name, download_url))

        return models
    except Exception as e:
        return str(e)
    
def download_files_with_aria2c(models, download_directory="/root/autodl-tmp/upmod"):
    """
    Download files using aria2c.

    Args:
    models (list of tuples): List of model name and download URL tuples.
    download_directory (str): Path to the directory where files will be downloaded.
    """
    for model_name, url in models:
        # 构建完整的下载命令
        command = f'aria2c -s 16 -x 8 --seed-time=0 "{url}" -d "{download_directory}" -o "{model_name}"'
        cmd_run(command)

# Example usage
models = get_model_names_and_urls(base_url="https://huggingface.co/conrevo/AnimateDiff-A1111/tree/main/control")
# for model_name, url in models:
#     print(f"Model: {model_name}, URL: {url}")

download_files_with_aria2c(models, '/root/autodl-tmp/ancontrol')


01/08 22:46:49 [NOTICE] Downloading 1 item(s)

01/08 22:46:50 [NOTICE] CUID#7 - Redirecting to https://cdn-lfs-us-1.huggingface.co/repos/eb/36/eb3682f47ef4d8ca29a706b4dac4d42561a0670a17f803febf9feb06ed6286ea/2d07d75bd0395fd419477395305b34d4cd23b5d3e7ad5896d73b7fdfe49594d2?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27mm_sd15_v3_sparsectrl_rgb.safetensors%3B+filename%3D%22mm_sd15_v3_sparsectrl_rgb.safetensors%22%3B&Expires=1704984410&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwNDk4NDQxMH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2ViLzM2L2ViMzY4MmY0N2VmNGQ4Y2EyOWE3MDZiNGRhYzRkNDI1NjFhMDY3MGExN2Y4MDNmZWJmOWZlYjA2ZWQ2Mjg2ZWEvMmQwN2Q3NWJkMDM5NWZkNDE5NDc3Mzk1MzA1YjM0ZDRjZDIzYjVkM2U3YWQ1ODk2ZDczYjdmZGZlNDk1OTRkMj9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Signature=OLuQ4x-EeOAtpkp4RkC1Dm0m2PVFws%7EoryK0ArJP3jjGNmz11FJJulyxlrbYmWX8qbvxs-RN9wAqTFkDnOq4-ZnMXxdy7BZHOQlorJTZZrqup0eys94updVeiG2Exa